In [1]:
import numpy as np
import json
import sys
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest


import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
sys.path.insert(0, 'src')
from helper import *
from eda import *
from train import *                       
from etl import *

%load_ext autoreload

%autoreload 2

### etl (new readfilerun and gen)

In [4]:
df = readfilerun_simple('data/raw/train_r/20220116T055105_20-100-true-20-100-iperf.csv')
df10 = agg10(genfeat(df))


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,total_pkts_min,total_pkts_max,...,pkt sum,longest_seq,longest_seq_min,longest_seq_max,total_pkt_sizes,byte_ratio,mean_tdelta,max_tdelta,loss,latency
0,370373.2,7110.925503,6.0,363000.0,7373.2,242.0,134.8,376.8,216.0,685.0,...,370373.2,6.7,3.0,8.0,370373.2,1083.537122,2.778182,40.5,100.0,20.0
1,468856.8,6551.301672,6.0,462750.0,6106.8,308.5,114.9,423.4,242.0,583.0,...,468856.8,7.7,6.0,14.0,468856.8,1083.537122,2.400794,43.7,100.0,20.0
2,381860.0,2961.806926,6.0,376800.0,5060.0,251.2,94.6,345.8,262.0,441.0,...,381860.0,7.2,6.0,9.0,381860.0,1083.537122,2.844551,43.4,100.0,20.0
3,376082.8,4210.239877,6.0,371100.0,4982.8,247.4,93.2,340.6,252.0,491.0,...,376082.8,7.1,6.0,10.0,376082.8,1083.537122,2.924540,43.6,100.0,20.0
4,403559.6,3621.508194,6.0,398100.0,5459.6,265.4,102.5,367.9,230.0,495.0,...,403559.6,7.4,5.0,10.0,403559.6,1083.537122,2.745876,43.2,100.0,20.0
5,338413.6,3993.754591,6.0,333900.0,4513.6,222.6,83.9,306.5,239.0,362.0,...,338413.6,6.7,5.0,9.0,338413.6,1083.537122,3.182794,43.1,100.0,20.0
6,341859.6,3721.544267,6.0,337350.0,4509.6,224.9,84.2,309.1,230.0,450.0,...,341859.6,6.6,5.0,8.0,341859.6,1083.537122,3.212894,43.2,100.0,20.0
7,428360.8,7087.863054,6.0,422700.0,5660.8,281.8,106.4,388.2,208.0,634.0,...,428360.8,7.0,5.0,10.0,428360.8,1083.537122,2.713966,43.3,100.0,20.0
8,340528.4,4347.346143,6.0,336000.0,4528.4,224.0,84.5,308.5,211.0,480.0,...,340528.4,6.7,5.0,8.0,340528.4,1083.537122,3.335741,43.4,100.0,20.0
9,353285.6,4755.075249,6.0,348600.0,4685.6,232.4,87.6,320.0,239.0,449.0,...,353285.6,7.1,6.0,10.0,353285.6,1083.537122,3.122712,43.3,100.0,20.0


In [9]:
df['group'] = df.index // 10
df.groupby('group').agg()

,Time,IP1,Port1,IP2,Port2,Proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,...,longest_seq,mean_tdelta,max_tdelta,number_ms,max_bytes,total_pkt_sizes,pkt_ratio,time_spread,byte_ratio,group
0,1642312265,172.21.0.3,40878,172.19.0.2,5001,6,553500,18292,369,316,...,8,1.008772,29,137,19628,571792,834.732847,690,834.732847,0
1,1642312266,172.21.0.3,40878,172.19.0.2,5001,6,235500,8432,157,155,...,4,3.135048,27,79,9260,243932,781.833333,975,781.833333,0
2,1642312267,172.21.0.3,40878,172.19.0.2,5001,6,289500,9988,193,186,...,3,2.574074,22,62,9364,299488,790.205805,973,790.205805,0
3,1642312268,172.21.0.3,40878,172.19.0.2,5001,6,405000,9228,270,173,...,8,2.235294,22,63,13708,414228,935.051919,988,935.051919,0
5,1642312269,172.21.0.3,40878,172.19.0.2,5001,6,232500,3364,155,61,...,8,4.562791,90,46,12052,235864,1091.962963,981,1091.962963,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,1642312560,172.21.0.3,40878,172.19.0.2,5001,6,309000,4152,206,78,...,6,3.286219,43,26,16796,313152,1102.647887,930,1102.647887,30
302,1642312561,172.21.0.3,40878,172.19.0.2,5001,6,237000,3576,158,66,...,6,4.372197,43,35,13708,240576,1074.000000,975,1074.000000,30
303,1642312562,172.21.0.3,40878,172.19.0.2,5001,6,366000,4896,244,93,...,6,2.901786,43,27,21312,370896,1100.581602,975,1100.581602,30
304,1642312563,172.21.0.3,40878,172.19.0.2,5001,6,316500,4548,211,84,...,6,3.176871,43,28,21260,321048,1088.298305,934,1088.298305,30


In [21]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [24]:
indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

20

['total_bytes',
 'max_bytes',
 '2->1Bytes',
 '2->1Pkts',
 'total_pkts',
 'number_ms',
 'pkt_ratio',
 'time_spread',
 'pkt sum',
 'longest_seq',
 'total_pkt_sizes',
 'byte_ratio']

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

label_col = 'latency'

df = pd.read_csv('outputs\\combined_transform.csv')

indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

X_train, X_test, y_train, y_test = train_test_split(
    df[[x for x in indexcol if x in df.columns]], df[label_col])
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
etree = ExtraTreesRegressor(n_estimators=400, n_jobs=4)
etreeft = etree.fit(X_train,y_train)

y_pred3 = etree.predict(X_test)
acc3= mean_squared_error(y_test, y_pred3)

print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

(5780, 12) (1927, 12) (5780,) (1927,)
mse: 1709.884903347172, r2: 0.9318839253945324


byte_ratio         0.553971
time_spread        0.102298
number_ms          0.075867
pkt_ratio          0.072764
max_bytes          0.032258
2->1Pkts           0.025018
total_pkt_sizes    0.024187
2->1Bytes          0.023896
total_bytes        0.023580
pkt sum            0.023410
total_pkts         0.023066
longest_seq        0.019684
dtype: float64

In [32]:
acc3

1888.4178210949665

In [44]:
print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

mse: 1888.4178210949665, r2: 0.9239026238224772


byte_ratio         0.570085
time_spread        0.099886
pkt_ratio          0.069825
number_ms          0.069792
max_bytes          0.028858
pkt sum            0.024635
total_pkts         0.023934
total_pkt_sizes    0.023690
2->1Bytes          0.022993
2->1Pkts           0.022918
total_bytes        0.022051
longest_seq        0.021332
dtype: float64

In [43]:
pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)


byte_ratio         0.570085
time_spread        0.099886
pkt_ratio          0.069825
number_ms          0.069792
max_bytes          0.028858
pkt sum            0.024635
total_pkts         0.023934
total_pkt_sizes    0.023690
2->1Bytes          0.022993
2->1Pkts           0.022918
total_bytes        0.022051
longest_seq        0.021332
dtype: float64